In [8]:
import openml
openml.config.apikey = openml.config.get_config_as_dict()['apikey']

In [98]:
d = openml.datasets.get_dataset(128)

In [125]:
from pandas import CategoricalDtype
import pandas as pd
import numpy as np

def get_target_col_type(dataset, target_col_name):
    try:
        if dataset.features:
            return next((feature.data_type for feature in dataset.features.values() if feature.name == target_col_name), None)
    except Exception as e:
        print(e)
        return None
def check_if_api_key_is_valid():
    if not openml.config.get_config_as_dict()['apikey']:
        print("API key is not set. Please set the API key using openml.config.apikey = 'your-key'")
        return False
    else:
        return True


def try_create_task(dataset_id):
    try:
        dataset = openml.datasets.get_dataset(dataset_id)
        target_col_name = dataset.default_target_attribute
        target_col_type = get_target_col_type(dataset, target_col_name)

        if target_col_type:
            if target_col_type in ['nominal', 'string', 'categorical']:
                evaluation_measure="predictive_accuracy"
                task_type = openml.tasks.TaskType.SUPERVISED_CLASSIFICATION
            elif target_col_type == 'numeric':
                evaluation_measure="mean_absolute_error"
                task_type = openml.tasks.TaskType.SUPERVISED_REGRESSION
            else:
                return None
            
            task = openml.tasks.create_task(
                dataset_id=dataset_id,
                task_type=task_type,
                target_name=target_col_name,
                evaluation_measure=evaluation_measure,
                estimation_procedure_id=1)
            # try:
            if check_if_api_key_is_valid():
                task.publish()
            else:
                return None
            print(f"Task created: {task}, task_id: {task.task_id}")
            return task.task_id
        else:
            return None
    except Exception as e:
        print(e)
    
    return None



In [128]:
try_create_task(46342)

Task created: OpenML Classification Task
Task Type Description: https://www.openml.org/tt/TaskType.SUPERVISED_CLASSIFICATION
Task ID..............: 362130
Task URL.............: https://www.openml.org/t/362130
Estimation Procedure.: None
Evaluation Measure...: predictive_accuracy
Target Feature.......: result
Cost Matrix..........: Available, task_id: 362130


362130

- todo
  - requirements to poetry
  - install all required frameworks
  - check if results exist

In [5]:
import sqlite3

In [13]:
# connect to the database and delete rows that have framework==autogluon
conn = sqlite3.connect('./data/runs.db')
c = conn.cursor()
c.execute("DELETE FROM runs WHERE framework='gama'")
conn.commit()

In [14]:
# select * from runs
c.execute("SELECT * FROM runs")
print(c.fetchall())


[(2, 2, 'randomforest'), (3, 3, 'randomforest'), (4, 4, 'randomforest'), (5, 5, 'randomforest'), (6, 6, 'randomforest'), (7, 7, 'randomforest'), (8, 8, 'randomforest'), (9, 9, 'randomforest'), (10, 10, 'randomforest'), (11, 11, 'randomforest')]


In [ ]:
import openml

In [4]:
openml.datasets.get_dataset(1)

OpenML Dataset
Name..........: anneal
Version.......: 2
Format........: ARFF
Upload Date...: 2014-04-06 23:19:20
Licence.......: Public
Download URL..: https://api.openml.org/data/v1/download/1/anneal.arff
OpenML URL....: https://www.openml.org/d/1
# of features.: 39
# of instances: 898

In [1]:
import openml
import pandas as pd
import os
import subprocess
from tqdm.auto import tqdm
import json

/Users/smukherjee/.pyenv/versions/3.9.19/envs/automlb/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class TaskConfig:
    def __init__(self, *args):
        self.args = args

In [23]:
class TaskFinder:
    def __init__(
        self,
        testing_mode=True,
        use_cache=True,
        run_mode="docker",
        num_tasks_to_return=1,
        save_every_n_tasks=10,
    ):
        self.testing_mode = testing_mode
        self.cache_file_name = "data/dataset_list.csv"
        self.global_results_store = {}
        self.num_tasks_to_return = num_tasks_to_return
        self.use_cache = use_cache
        self.save_every_n_tasks = save_every_n_tasks
        # self.benchmarks_to_use = ["autosklearn"]
        self.benchmarks_to_use = ["randomforest", "autogluon"]
        # Ensure required folders exist
        self.make_files(["data"])
        self.run_mode = run_mode

        self.check_run_mode()
        # Load datasets from cache or OpenML
        self.datasets = self.load_datasets()

        # If in testing mode, only take the first dataset
        if self.testing_mode:
            # self.datasets = self.datasets.head(1)
            # 43972 - tic tac
            self.datasets = self.datasets[self.datasets["did"] == 50]

    def check_run_mode(self):
        possible = ["local", "aws", "docker", "singularity"]
        if self.run_mode not in possible:
            raise ValueError(
                f"Invalid run mode: {self.run_mode}. Possible values are: {possible}"
            )

    def load_datasets(self):
        datasets = openml.datasets.list_datasets(output_format="dataframe")
        # if the cache file exists, append the new unique datasets to it
        if self.use_cache and os.path.exists(self.cache_file_name):
            cached_datasets = pd.read_csv(self.cache_file_name)
        else:
            if os.path.exists(self.cache_file_name):
                cached_datasets = pd.read_csv(self.cache_file_name)
                datasets = pd.concat([datasets, cached_datasets], ignore_index=True)
                datasets = datasets.drop_duplicates(subset=["did"])

                # Save the updated dataset list to the cache
                datasets.to_csv(self.cache_file_name)
        return datasets

    def make_files(self, folders):
        for folder in folders:
            os.makedirs(folder, exist_ok=True)

    def get_tasks_from_dataset(self, dataset_id, num_tasks_to_return=1):
        try:
            tasks = openml.tasks.list_tasks(
                data_id=dataset_id, output_format="dataframe"
            )
            # if the task column estimation_procedure is not 10-fold Crossvalidation drop the row
            tasks = tasks[tasks["estimation_procedure"] == "10-fold Crossvalidation"]
            # return the first num_tasks_to_return tasks
            tasks = tasks.head(num_tasks_to_return)
            return tasks["tid"].tolist() if not len(tasks) == 0 else None
        except Exception as e:
            print(f"Error retrieving tasks for dataset {dataset_id}: {e}")
            return None

    def run_all_benchmarks_on_task(self, task_id):
        # task_id = task_id.strip()
        for benchmark_type in self.benchmarks_to_use:
            try:
                result = subprocess.run(
                    [
                        "python3",
                        "automlbenchmark/runbenchmark.py",
                        benchmark_type,
                        f"openml/t/{task_id}",
                        "--mode",
                        self.run_mode,
                    ],
                    text=True,
                    capture_output=True,
                )
                print(result.stderr)
                # Filter and return relevant benchmark output
                if self.run_mode == "local":
                    return [
                        line for line in result.stderr.split("\n") if "TaskConfig" in line
                    ]
                elif self.run_mode == "docker":
                   return [
                        line for line in result.stderr.split("\n") if "Starting docker: docker run --name " in line
                    ] 
            except subprocess.CalledProcessError as e:
                print(f"Benchmark run failed for task {task_id}: {e}")
                return []

    def get_task_config_from_str(self, task_str):
        try:
            if self.run_mode == "local":
                return eval(task_str).__dict__["args"][0]
            elif self.run_mode == "docker":
                # --name randomforest.openml_t_49.test.docker.20241009T153843.sIiURqJG8Z99apxsQC8ISg__ 
                
        except Exception as e:
            print(f"Error parsing task config: {e}")
            return None

    def get_task_for_dataset(self, dataset_id):
        task_ids = self.get_tasks_from_dataset(dataset_id, self.num_tasks_to_return)
        if task_ids:
            for task_id in tqdm(
                task_ids, desc=f"Running benchmark on dataset {dataset_id}"
            ):
                benchmark_results = self.run_all_benchmarks_on_task(
                    task_id,
                )
                for result in tqdm(
                    benchmark_results, desc=f"Processing results for task {task_id}"
                ):
                    task_config = self.get_task_config_from_str(result)
                    if task_config and self.run_mode == "local":
                        current_run_info = {
                            "dataset_id": dataset_id,
                            "task_id": task_id,
                            "task_config": task_config,
                        }
                        self.global_results_store[dataset_id] = current_run_info
                    if task_config and self.run_mode == "docker":
                        current_run_info = {
                            "dataset_id": dataset_id,
                            "task_id": task_id,
                        }
                # write the results to a file every 10 tasks
                if len(self.global_results_store) % self.save_every_n_tasks == 0:
                    self.write_global_to_file()

    def write_global_to_file(
        self, file_name="data/links_to_automl_files_per_dataset.json"
    ):
        # read the current file if it exists and append the new results
        if os.path.exists(file_name):
            with open(file_name, "r") as f:
                data = json.load(f)
                data.update(self.global_results_store)
        else:
            data = self.global_results_store
        with open(file_name, "w") as f:
            json.dump(data, f)

    def load_global_from_file(
        self, file_name="data/links_to_automl_files_per_dataset.json"
    ):
        if os.path.exists(file_name):
            with open(file_name, "r") as f:
                self.global_results_store = json.load(f)
        else:
            print("No global results file found")

    def run_benchmark_on_all_datasets(self):
        for _, row in self.datasets.iterrows():
            dataset_id = row["did"]
            self.get_task_for_dataset(dataset_id)

        # save info
        self.write_global_to_file()

    def upload_results_to_openml(self):
        raise NotImplementedError()

In [28]:
tf = TaskFinder()

In [29]:
tf.run_benchmark_on_all_datasets()

Running benchmark on dataset 50:   0%|          | 0/1 [00:00<?, ?it/s]

/Users/smukherjee/.pyenv/versions/3.9.19/envs/automlb/lib/python3.9/site-packages/_distutils_hack/__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Running benchmark `randomforest` on `openml/t/49` framework in `docker` mode.
Loading frameworks definitions from ['/Users/smukherjee/Documents/CODE/Github/OpenML-auto-automl/automlbenchmark/resources/frameworks.yaml'].
Loading benchmark constraint definitions from ['/Users/smukherjee/Documents/CODE/Github/OpenML-auto-automl/automlbenchmark/resources/constraints.yaml'].
Loading openml task 49.
Running cmd `docker images -q automlbenchmark/randomforest:stable-dev`
Running cmd `docker pull automlbenchmark/randomforest:stable-dev`
Error response from daemon: manifest for automlbenchmark/randomforest:stable-

Running benchmark on dataset 50: 100%|██████████| 1/1 [00:41<00:00, 41.49s/it]


In [3]:
from pathlib import Path
store = tf.global_results_store
for datasets in store.keys():
    result_file = Path(store[datasets]["task_config"]["output_dir"])/"scores/results.csv"
    pandas_file = pd.read_csv(result_file)
    print(pandas_file.head())
    break

NameError: name 'tf' is not defined

In [30]:
pandas_file.columns

Index(['id', 'task', 'framework', 'constraint', 'fold', 'type', 'result',
       'metric', 'mode', 'version', 'params', 'app_version', 'utc', 'duration',
       'training_duration', 'predict_duration', 'models_count', 'seed', 'info',
       'acc', 'auc', 'balacc', 'logloss'],
      dtype='object')

In [14]:
import pandas as pd
from glob import glob
from pathlib import Path
import json

{'framework': 'RandomForest', 'framework_params': {'n_estimators': 2000}, 'framework_version': '1.0', 'type': 'classification', 'name': 'anneal', 'fold': 1, 'metric': 'logloss', 'metrics': ['logloss', 'acc', 'balacc'], 'seed': 1095784781, 'job_timeout_seconds': 1200, 'max_runtime_seconds': 600, 'cores': 4, 'max_mem_size_mb': 10721, 'min_vol_size_mb': -1, 'input_dir': '/input', 'output_dir': '/output/', 'output_predictions_file': '/output/predictions/anneal/1/predictions.csv', 'ext': {}, 'type_': 'multiclass', 'output_metadata_file': '/output/predictions/anneal/1/metadata.json'}


In [1]:
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

In [32]:
# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Layout
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1("Model Results Dashboard"), width=12)
    ]),
    
    dbc.Row([
        dbc.Col([
            html.Label("Select Metric:"),
            dcc.Dropdown(
                id='metric-dropdown',
                options=[
                    {'label': 'Accuracy', 'value': 'acc'},
                    {'label': 'AUC', 'value': 'auc'},
                    {'label': 'Log Loss', 'value': 'logloss'}
                ],
                value='acc',
                clearable=False
            )
        ], width=4),
    ]),

    dbc.Row([
        dbc.Col(dcc.Graph(id='metric-graph'), width=12)
    ]),

    dbc.Row([
        dbc.Col(dcc.Graph(id='task-pie-chart'), width=6),
        dbc.Col(dcc.Graph(id='framework-bar-chart'), width=6),
    ]),

], fluid=True)

# Callbacks
@app.callback(
    Output('metric-graph', 'figure'),
    Output('task-pie-chart', 'figure'),
    Output('framework-bar-chart', 'figure'),
    Input('metric-dropdown', 'value')
)


# Run app and display result inline in the notebook
app.run_server(mode='inline')

In [40]:
import re
def get_task_id_from_folder_name(folder_name):
    name = re.findall(pattern=r"_t_.*[0-9]\.", string=folder_name)
    print(name)
    if isinstance(name, list) and len(name) > 0:
        return str(name[0])
    else:
        return name

In [41]:
get_task_id_from_folder_name(folder_name="randomforest.openml_t_5.test.docker.20241016T141320")

['_t_5.']


'_t_5.'

In [42]:
import openml

In [51]:
t = openml.tasks.get_task(10, download_data=False, download_qualities= False)
t.dataset_id

10

In [ ]:
i